In [8]:
import numpy as np

def logistic_regression_dataset_synthesis(N, d):
    w = np.random.randn(d)
    X = np.random.randn(N, d)
    logits = X @ w
    probs = 1 / (1 + np.exp(-logits))
    y = (np.random.rand(N) < probs).astype(np.float32)
    return X, y, w

In [9]:
from sklearn.model_selection import train_test_split
X, y, w= logistic_regression_dataset_synthesis(1000, 10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, random_state=42, fit_intercept=False)
model.fit(X_train, y_train)
train_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

Train Accuracy: 0.8850, Test Accuracy: 0.9200


In [11]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.decomposition import PCA


# bank_marketing = pd.read_csv(os.path.join('.', 'bank', 'bank-full.csv'), sep=";")
# # data (as pandas dataframes)
# # DataFrame of features (categorical + numeric)
# X = bank_marketing.drop(columns=["y"])
# y = bank_marketing["y"]                  # Series target: "yes"/"no"
# y = (y.astype(str).str.lower() == "yes").astype(int).to_numpy()

# cat_cols = X.select_dtypes(include=["object", "category"]).columns
# num_cols = X.columns.difference(cat_cols)

# X = X.drop(columns=cat_cols)

# ct = ColumnTransformer(
#     transformers=[
#         ("num", StandardScaler(with_mean=True, with_std=True), list(num_cols)),
#         # ("cat", Pipeline([
#         #     ("onehot", OneHotEncoder(
#         #         handle_unknown="ignore", sparse_output=False)),
#         #     ("scaler", StandardScaler(with_mean=True, with_std=False)),
#         # ]),
#         #     list(cat_cols)),
#     ],
#     remainder="drop",
# )

# def pca_wrapper(X_train, X_test, whiten, tol=1e-6):
#     pca = PCA(whiten=False, random_state=42)
#     X_train_pca = pca.fit_transform(X_train)
#     # get the non-zero eigenvalue components
#     non_zero_var_indices = pca.explained_variance_ > tol
#     X_train_pca = X_train_pca[:, non_zero_var_indices]
#     X_test_pca = pca.transform(X_test)
#     X_test_pca = X_test_pca[:, non_zero_var_indices]
#     if whiten:
#         X_train_whitened = X_train_pca / np.sqrt(pca.explained_variance_[non_zero_var_indices])
#         X_test_whitened = X_test_pca / np.sqrt(pca.explained_variance_[non_zero_var_indices])
#         return X_train_whitened, X_test_whitened
#     else:
#         return X_train_pca, X_test_pca

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y)
# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)
# # X_train, X_test = pca_wrapper(X_train, X_test, whiten=True)
# # pca = PCA(whiten=False, random_state=42, n_components=0.99)
# # X_train = pca.fit_transform(X_train)
# # X_test = pca.transform(X_test)

In [12]:
from scipy.special import gammaln

def exact_var_1d(c):
    """
    Exact variance of
        Y = (sum_i m_i * c_i) / (sum_i m_i),
    where m_i ~ Bernoulli(0.5) i.i.d., and Y=0 when sum_i m_i = 0.

    Uses log-sum-exp vectorization for numerical stability and speed.
    """
    c = np.asarray(c, dtype=float)
    n = c.size
    a = np.sum(c)
    b = np.sum(c**2)

    # ----- compute E_invK = E[1/K * 1_{K≥1}] -----
    k = np.arange(1, n + 1)
    # log(C(n,k)) = log(n!) - log(k!) - log((n-k)!)
    log_comb = gammaln(n + 1) - gammaln(k + 1) - gammaln(n - k + 1)
    log_terms = log_comb - np.log(k) - n * np.log(2)

    # log-sum-exp for numerical stability
    max_log = np.max(log_terms)
    E_invK = np.exp(max_log) * np.sum(np.exp(log_terms - max_log))

    # probability K ≥ 1
    p_nonzero = 1 - 2 ** (-n)

    # ----- compute variance -----
    term1 = (b * n - a**2) / (n**2 * (n - 1))
    var_conditional = term1 * (n * E_invK - p_nonzero)
    var_mean = (a / n)**2 * p_nonzero * (1 - p_nonzero)

    return var_conditional + var_mean

In [13]:
import numpy as np
from scipy.special import expit as sigmoid
from scipy.special import lambertw
from scipy.optimize import minimize

def optimal_eta(mu, T, C, e0, var):
    if e0 == 0:
        return 0.0
    
    if var == 0:
        # minimize (1-eta * mu)^T * e0, this is achieved at eta = 1/mu
        return 1/mu

    alpha = mu * T
    beta = (1+C) * var /  mu
    a = alpha * e0**2 / beta
    
    if a>500:
        print("Using approximation for large a")
        # eta = 1/alpha * (np.log(a) - np.log(a)/a) # very good approximation when a is large
        eta = 1/alpha * (np.log1p(a) - np.log1p(a)/(a+1)) # very good approximation when a is large
    else:
        eta = 1/alpha * (a+1 - lambertw(np.exp(a+1)).real)
        
    return eta

def pac_private_logistic_regression(X, y, mu, mi_budget, T=10, privacy_aware=True):

    def objective(w):
        N = X.shape[0]
        z = X @ w
        y_hat = sigmoid(z)
        loss = -(1/N) * np.sum(y * np.log(y_hat) + (1 - y) * np.log1p(- y_hat))
        reg_term = (mu / 2) * np.sum(w ** 2)
        return loss + reg_term
    
    e0 = minimize(objective, np.zeros(X.shape[1]), method='L-BFGS-B', tol=1e-10).x

    N, d = X.shape
    w = np.zeros(d)
    losses = []
    # C = d * T / 2.0 / mi_budget
    C = 0

    for _ in range(T):
        z = X @ w
        y_hat = sigmoid(z)

        # --- Per-sample gradients ---
        per_sample_grads = (y_hat - y)[:, np.newaxis] * X  # shape (N, d)

        for d_i in range(d):
            
            grad_i_var = exact_var_1d(per_sample_grads[:, d_i])
            # grad_i_var = np.var(np.array([per_sample_grads[np.random.rand(N) < 0.5, d_i].mean() for _ in range(128)])) # scalar
            grad = per_sample_grads[np.random.rand(N) < 0.5, d_i].mean() + mu * w[d_i] + np.sqrt(C * grad_i_var) * np.random.randn()

            lr = optimal_eta(mu=mu, T=T, C=C if privacy_aware else 0, e0=e0[d_i], var=grad_i_var)
            # lr = 0.1

            w[d_i] = w[d_i] - lr * grad
        
        z = X @ w
        y_hat = sigmoid(z)
        loss = -(1/N) * np.sum(y * np.log(y_hat) + (1 - y) * np.log1p(- y_hat))

        reg_term = (mu / 2) * np.sum(w ** 2)
        losses.append(loss + reg_term)
        print(f'Train loss: {losses[-1]:.4f}')

    return w, losses

def evaluate_model(X, y, w):
    z = X @ w
    y_hat = sigmoid(z)
    y_pred = (y_hat > 0.5).astype(int)
    accuracy = np.mean(y_pred == y)
    return accuracy

def model_loss(X, y, w, mu):
    N = X.shape[0]
    z = X @ w
    y_hat = sigmoid(z)
    loss = -(1/N) * np.sum(y * np.log(y_hat) + (1 - y) * np.log1p(- y_hat))
    reg_term = (mu / 2) * np.sum(w ** 2)
    return loss + reg_term

In [14]:
accs = []
for _ in range(10):
    w, losses = pac_private_logistic_regression(X_train, y_train, mu=1e-3, mi_budget=1/1024, T=50, privacy_aware=True)
    test_accuracy = evaluate_model(X_test, y_test, w)
    accs.append(test_accuracy)

    print(f'Test accuracy: {test_accuracy:.4f}')

print(f'Average test accuracy over 10 runs: {np.mean(accs):.4f} ± {np.std(accs):.4f}')

Train loss: 0.3975
Train loss: 0.3634
Train loss: 0.3290
Train loss: 0.2957
Train loss: 0.2881
Train loss: 0.2801
Train loss: 0.2841
Train loss: 0.2731
Train loss: 0.2879
Train loss: 0.2814
Train loss: 0.2908
Train loss: 0.2681
Train loss: 0.2665
Train loss: 0.2708
Train loss: 0.2733
Train loss: 0.2713
Train loss: 0.2672
Train loss: 0.2635
Train loss: 0.2578
Train loss: 0.2569
Train loss: 0.2582
Train loss: 0.2584
Train loss: 0.2594
Train loss: 0.2576
Train loss: 0.2599
Train loss: 0.2773
Train loss: 0.2722
Train loss: 0.2700
Train loss: 0.2600
Train loss: 0.2608
Train loss: 0.2692
Train loss: 0.2616
Train loss: 0.2609
Train loss: 0.2592
Train loss: 0.2578
Train loss: 0.2567
Train loss: 0.2565
Train loss: 0.2623
Train loss: 0.2622
Train loss: 0.2588
Train loss: 0.2568
Train loss: 0.2578
Train loss: 0.2577
Train loss: 0.2603
Train loss: 0.2574
Train loss: 0.2606
Train loss: 0.2603
Train loss: 0.2560
Train loss: 0.2578
Train loss: 0.2578
Test accuracy: 0.9150
Train loss: 0.4186
Train los